# Merges

Merge lets us combine multiple data frames. The general syntax is as follows to merge pandas dataframes df_1 and df_2:
    
    df_1.merge(df_2, how = ["inner", "outer", "left"] , left_on = left_column_name, right_on = right_column_name)

the example above shows the three main inputs to the how arguments.  When actually using a merge you should only specify one of these three values for the how arguments. The how argument specifies the type of merge and the left_on and right_on arguments tells pandas which columns to match up from each of the two dataframes when deciding which rows to keep.  In the example above, df_1 is the left dataframe and df_2 is the right dataframe.

In [2]:
import pandas as pd

#Create doctor df

doctors = pd.DataFrame({"doctor_id":[210,211,212,213],"doc_name":["Jon", "Peter", "Ke", "Pat"],\
                       "degree":["MD","MBBS", "MD", "MD"]})

#Create visits
visits = pd.DataFrame({"doctor_id":[210,214,215,212,212], "patient_name": ["Julia", "TJ", "John", "James", "Jason"],\
                      "year":[2015,2014, 2015, 2016, 2012]})

doctors

,degree,doc_name,doctor_id
0,MD,Jon,210
1,MBBS,Peter,211
2,MD,Ke,212
3,MD,Pat,213


In [3]:
visits

,doctor_id,patient_name,year
0,210,Julia,2015
1,214,TJ,2014
2,215,John,2015
3,212,James,2016
4,212,Jason,2012


For every visit, I would like to create a table with the doctor that saw the patient, if one exists. We will merge on the doctor_id column with an inner merge.  With an inner merge, if a match isn't found, then we do not include the row. If the two columns that you want to merge on have the same name, then you can simply specify the on argument instead of separate left_on and right_on arguments taking the same value.

In [5]:
#Basic inner merge
visits.merge(doctors, how  = "inner", on  = "doctor_id" )

,doctor_id,patient_name,year,degree,doc_name
0,210,Julia,2015,MD,Jon
1,212,James,2016,MD,Ke
2,212,Jason,2012,MD,Ke


Notice that we have lost the visits of TJ and John. This is because the doctors table does not have a doctor corresponding to ids 214 and 215.  In the case, the inner merge will not include these visits in the dataframe return from the merge. We can slice the returned dataframe using .loc to get the desired columns or remove duplicates that arise because of the merge. 

In [7]:
visits.merge(doctors, how  = "inner", on  = "doctor_id" ).loc[:,["patient_name", "year", "doc_name"]]

,patient_name,year,doc_name
0,Julia,2015,Jon
1,James,2016,Ke
2,Jason,2012,Ke


Now lets see how we can use a left merge to keep these visits that don't have a matching doctor.

In [9]:
#Basic left merge
visits.merge(doctors, how  = "left", on  = "doctor_id" ).loc[:,["patient_name", "year", "doc_name"]]

,patient_name,year,doc_name
0,Julia,2015,Jon
1,TJ,2014,NaN
2,John,2015,NaN
3,James,2016,Ke
4,Jason,2012,Ke


Notice that in this example, we get a row for every visit, but for visits that don't have a matching doctor_id we get NaNs from the doctors table.

Finally, let's see an example of an outer merge.  Consider the following two dataframes.

In [3]:
#Create Two data frames
df1 = pd.DataFrame({ "RushPoints":[12,32,34,12], "Week":[1,2,3,4]})
df1 = df1[["Week", "RushPoints"]]
df1

,Week,RushPoints
0,1,12
1,2,32
2,3,34
3,4,12


In [4]:
df2 = pd.DataFrame({"Week":[2,3,5], "RecPoints":[101,310,234]})
df2 = df2[["Week", "RecPoints"]]
df2

,Week,RecPoints
0,2,101
1,3,310
2,5,234


Lets say I wanted to combine the records and put a zero if one of the points categories did not exists for the given week. In this case I want to merge on Week, but each table has weeks that other doesn't have so a left merge will delete rows no matter which dataframe I choose as the left one. 

In [14]:
#Basic outer merge
final = df1.merge(df2, how = "outer", on = "Week")
final

,Week,RushPoints,RecPoints
0,1,12.0,NaN
1,2,32.0,101.0
2,3,34.0,310.0
3,4,12.0,NaN
4,5,NaN,234.0


Notice that the resulting data frame has a row for each week!

In [35]:
#Replace the NAN with 0
final.fillna(0, inplace = True)
final

,Week,RushPoints,RecPoints
0,1,12.0,0.0
1,2,32.0,101.0
2,3,34.0,310.0
3,4,12.0,0.0
4,5,0.0,234.0
